# Product Image Generation with OpenAI DALL-E

This notebook generates high-quality product images using OpenAI's DALL-E image generation API. We'll read product data from the CSV file and create realistic product images based on the product specifications including name, brand, category, color, and description.

## Requirements
- OpenAI API key
- Python packages: openai, pandas, requests, pillow
- Product data in CSV format

## Output
Images will be saved in the current directory with filenames based on product ID and name.

In [1]:
# Import required libraries
import pandas as pd
import os
import requests
from openai import OpenAI
from PIL import Image
import io
import time
from typing import List, Dict
import json

# Set up OpenAI client
# Make sure to set your OpenAI API key as an environment variable: OPENAI_API_KEY
client = OpenAI()

print("Libraries imported successfully!")
print("OpenAI client initialized")

Libraries imported successfully!
OpenAI client initialized


In [2]:
# Load product data from CSV
products_df = pd.read_csv('../products.csv')

print("Product data loaded successfully!")
print(f"Number of products: {len(products_df)}")
print("\nFirst few products:")
print(products_df.head())

print("\nColumn information:")
print(products_df.info())

print("\nUnique categories:")
print(products_df['category'].unique())

print("\nUnique brands:")
print(products_df['brand'].unique())

Product data loaded successfully!
Number of products: 10

First few products:
   id           name        brand   category     color  \
0   1    Ceramic Mug         Muji  Wearables     Olive   
1   2     Wall Clock      Normann  Wearables     White   
2   3  Leather Pouch         Muji  Wearables  Charcoal   
3   4    Ceramic Mug  Ferm Living       Home   Natural   
4   5    Linen Throw      Normann  Wearables   Natural   

                                         description   price  units_sold  
0  A premium ceramic mug crafted with attention t...  194.33         323  
1  A premium wall clock crafted with attention to...  272.10         402  
2  A premium leather pouch crafted with attention...   32.48         164  
3  A premium ceramic mug crafted with attention t...  113.76         156  
4  A premium linen throw crafted with attention t...   66.36         213  

Column information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 

In [3]:
def create_image_prompt(product_row: pd.Series) -> str:
    """
    Create a detailed image generation prompt based on product information.
    
    Args:
        product_row: A pandas Series containing product information
    
    Returns:
        str: A detailed prompt for image generation
    """
    name = product_row['name']
    brand = product_row['brand']
    category = product_row['category']
    color = product_row['color']
    description = product_row['description']
    
    # Create a detailed prompt that incorporates all product information
    prompt = f"""
    A professional product photography of a {color.lower()} {name.lower()} by {brand}, 
    designed for {category.lower()} category. {description} 
    The product should be photographed on a clean, minimalist background with professional lighting. 
    High-quality, commercial product shot, studio lighting, clean aesthetic, 
    modern design, premium materials, photorealistic.
    """.strip().replace('\n', ' ').replace('  ', ' ')
    
    return prompt

def generate_product_image(product_row: pd.Series, output_dir: str = '.') -> Dict:
    """
    Generate an image for a product using OpenAI's DALL-E API.
    
    Args:
        product_row: A pandas Series containing product information
        output_dir: Directory to save the generated image
    
    Returns:
        dict: Information about the generated image
    """
    try:
        # Create the prompt
        prompt = create_image_prompt(product_row)
        product_id = product_row['id']
        product_name = product_row['name'].replace(' ', '_').lower()
        
        print(f"\nGenerating image for Product {product_id}: {product_row['name']}")
        print(f"Prompt: {prompt[:100]}...")
        
        # Generate image using OpenAI API
        response = client.images.generate(
            model="dall-e-3",  # Use DALL-E 3 for highest quality
            prompt=prompt,
            size="1024x1024",  # High resolution
            quality="standard",  # Can be "standard" or "hd"
            n=1  # Number of images to generate
        )
        
        # Get the image URL
        image_url = response.data[0].url
        
        # Download and save the image
        image_response = requests.get(image_url)
        image = Image.open(io.BytesIO(image_response.content))
        
        # Create filename
        filename = f"{product_id}.png"
        filepath = os.path.join(output_dir, filename)
        
        # Save the image
        image.save(filepath)
        
        result = {
            'product_id': product_id,
            'product_name': product_row['name'],
            'filename': filename,
            'filepath': filepath,
            'prompt': prompt,
            'status': 'success',
            'image_url': image_url
        }
        
        print(f"✅ Image saved: {filename}")
        return result
        
    except Exception as e:
        print(f"❌ Error generating image for product {product_row['id']}: {str(e)}")
        return {
            'product_id': product_row['id'],
            'product_name': product_row['name'],
            'filename': None,
            'filepath': None,
            'prompt': prompt if 'prompt' in locals() else None,
            'status': 'error',
            'error': str(e)
        }

print("Helper functions defined successfully!")

Helper functions defined successfully!


In [4]:
# Let's first generate and review sample prompts for the first few products
print("Sample prompts for the first 3 products:")
print("=" * 50)

for idx, (_, product) in enumerate(products_df.head(3).iterrows()):
    prompt = create_image_prompt(product)
    print(f"\nProduct {product['id']}: {product['name']} by {product['brand']}")
    print(f"Category: {product['category']}, Color: {product['color']}")
    print(f"Prompt: {prompt}")
    print("-" * 50)

Sample prompts for the first 3 products:

Product 1: Ceramic Mug by Muji
Category: Wearables, Color: Olive
Prompt: A professional product photography of a olive ceramic mug by Muji,   designed for wearables category. A premium ceramic mug crafted with attention to detail.   The product should be photographed on a clean, minimalist background with professional lighting.   High-quality, commercial product shot, studio lighting, clean aesthetic,   modern design, premium materials, photorealistic.
--------------------------------------------------

Product 2: Wall Clock by Normann
Category: Wearables, Color: White
Prompt: A professional product photography of a white wall clock by Normann,   designed for wearables category. A premium wall clock crafted with attention to detail.   The product should be photographed on a clean, minimalist background with professional lighting.   High-quality, commercial product shot, studio lighting, clean aesthetic,   modern design, premium materials, photo

In [5]:
# Check if OpenAI API key is set
api_key_set = bool(os.getenv('OPENAI_API_KEY'))
print(f"OpenAI API key set: {api_key_set}")

if not api_key_set:
    print("⚠️  WARNING: OpenAI API key not found!")
    print("Please set your OpenAI API key as an environment variable:")
    print("export OPENAI_API_KEY='your-api-key-here'")
    print("\nOr set it in this notebook:")
    print("import os")
    print("os.environ['OPENAI_API_KEY'] = 'your-api-key-here'")
else:
    print("✅ API key is configured")
    
    # Generate a test image for the first product
    print("\n🧪 Generating test image for the first product...")
    test_result = generate_product_image(products_df.iloc[0])
    
    if test_result['status'] == 'success':
        print("✅ Test image generation successful!")
        print(f"Test image saved as: {test_result['filename']}")
    else:
        print("❌ Test image generation failed")
        print(f"Error: {test_result.get('error', 'Unknown error')}")

OpenAI API key set: True
✅ API key is configured

🧪 Generating test image for the first product...

Generating image for Product 1: Ceramic Mug
Prompt: A professional product photography of a olive ceramic mug by Muji,   designed for wearables category...
✅ Image saved: 1.png
✅ Test image generation successful!
Test image saved as: 1.png


In [6]:
# Generate images for all products
if api_key_set:
    print("🚀 Starting image generation for all products...")
    print(f"Total products to process: {len(products_df)}")
    print("=" * 60)
    
    results = []
    
    for idx, (_, product) in enumerate(products_df.iterrows()):
        print(f"\nProgress: {idx + 1}/{len(products_df)}")
        
        # Generate image for current product
        result = generate_product_image(product)
        results.append(result)
        
        # Add a small delay to avoid rate limiting
        if idx < len(products_df) - 1:  # Don't wait after the last image
            print("⏳ Waiting 2 seconds to avoid rate limiting...")
            time.sleep(2)
    
    print("\n" + "=" * 60)
    print("📊 GENERATION SUMMARY")
    print("=" * 60)
    
    # Summary statistics
    successful = sum(1 for r in results if r['status'] == 'success')
    failed = sum(1 for r in results if r['status'] == 'error')
    
    print(f"✅ Successful: {successful}")
    print(f"❌ Failed: {failed}")
    print(f"📊 Success rate: {successful/len(results)*100:.1f}%")
    
    # Save results to JSON for reference
    with open('image_generation_results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    print(f"\n💾 Results saved to: image_generation_results.json")
    
    # Show failed generations if any
    if failed > 0:
        print("\n❌ Failed generations:")
        for result in results:
            if result['status'] == 'error':
                print(f"  - Product {result['product_id']}: {result['product_name']} - {result['error']}")
else:
    print("⚠️  Skipping image generation - API key not set")

🚀 Starting image generation for all products...
Total products to process: 10

Progress: 1/10

Generating image for Product 1: Ceramic Mug
Prompt: A professional product photography of a olive ceramic mug by Muji,   designed for wearables category...
✅ Image saved: 1.png
⏳ Waiting 2 seconds to avoid rate limiting...

Progress: 2/10

Generating image for Product 2: Wall Clock
Prompt: A professional product photography of a white wall clock by Normann,   designed for wearables catego...
✅ Image saved: 2.png
⏳ Waiting 2 seconds to avoid rate limiting...

Progress: 3/10

Generating image for Product 3: Leather Pouch
Prompt: A professional product photography of a charcoal leather pouch by Muji,   designed for wearables cat...
✅ Image saved: 3.png
⏳ Waiting 2 seconds to avoid rate limiting...

Progress: 4/10

Generating image for Product 4: Ceramic Mug
Prompt: A professional product photography of a natural ceramic mug by Ferm Living,   designed for home cate...
✅ Image saved: 4.png
⏳ Waiti

In [7]:
# Display information about generated images
if api_key_set and 'results' in locals():
    print("📁 Generated Image Files:")
    print("=" * 40)
    
    for result in results:
        if result['status'] == 'success':
            print(f"✅ {result['filename']}")
            # Check if file actually exists
            if os.path.exists(result['filename']):
                file_size = os.path.getsize(result['filename']) / 1024  # Size in KB
                print(f"   Size: {file_size:.1f} KB")
            else:
                print(f"   ⚠️  File not found!")
        else:
            print(f"❌ Product {result['product_id']}: {result['product_name']} - FAILED")
    
    # Create a summary DataFrame
    summary_df = pd.DataFrame([
        {
            'product_id': r['product_id'],
            'product_name': r['product_name'],
            'filename': r['filename'],
            'status': r['status']
        } for r in results
    ])
    
    print("\n📋 Generation Summary Table:")
    print(summary_df.to_string(index=False))
    
    # Save summary to CSV
    summary_df.to_csv('image_generation_summary.csv', index=False)
    print("\n💾 Summary saved to: image_generation_summary.csv")
else:
    print("No images were generated. Please set your OpenAI API key and run the generation cells.")

📁 Generated Image Files:
✅ 1.png
   Size: 1435.6 KB
✅ 2.png
   Size: 1527.9 KB
✅ 3.png
   Size: 1407.4 KB
✅ 4.png
   Size: 1407.0 KB
✅ 5.png
   Size: 1609.2 KB
✅ 6.png
   Size: 1871.7 KB
✅ 7.png
   Size: 1709.7 KB
✅ 8.png
   Size: 1587.0 KB
✅ 9.png
   Size: 1011.8 KB
✅ 10.png
   Size: 1251.7 KB

📋 Generation Summary Table:
 product_id   product_name filename  status
          1    Ceramic Mug    1.png success
          2     Wall Clock    2.png success
          3  Leather Pouch    3.png success
          4    Ceramic Mug    4.png success
          5    Linen Throw    5.png success
          6  Leather Pouch    6.png success
          7   Notebook Set    7.png success
          8   Notebook Set    8.png success
          9    Ceramic Mug    9.png success
         10 Aroma Diffuser   10.png success

💾 Summary saved to: image_generation_summary.csv


## Next Steps

### 1. Set up your OpenAI API Key
Before running the image generation cells, make sure to set your OpenAI API key:

```bash
export OPENAI_API_KEY='your-api-key-here'
```

Or set it directly in the notebook:
```python
import os
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'
```

### 2. Generated Files
After running the notebook, you'll have:
- **Product images**: `product_{id}_{name}.png` files
- **Generation results**: `image_generation_results.json`
- **Summary**: `image_generation_summary.csv`

### 3. Cost Considerations
- DALL-E 3 costs $0.040 per image (1024×1024)
- For 10 products = ~$0.40
- Consider using DALL-E 2 for lower costs ($0.020 per image)

### 4. Customization Options
- Modify the `create_image_prompt()` function to adjust image style
- Change image size (256x256, 512x512, 1024x1024)
- Adjust quality settings (standard vs hd)
- Add custom styling preferences

### 5. Error Handling
The notebook includes comprehensive error handling and will continue processing even if some images fail to generate.

### 6. Rate Limiting
The notebook includes automatic delays between requests to respect OpenAI's rate limits.